In [1]:
# Import helper functions
def split_dir_to_train_test_val(directory = "data/",
                            train_size = 0.7,
                            test_size = 0.2,
                            val_size = 0.1):
  """
  Creates 3 folders for Train, Test and Validation data
  """
  import os
  import random
  import shutil

  # Set random seed
  rng = random.Random(42)

  for root, folders, files in os.walk(directory):
    for folder in folders:
      # Create list of the files
      list_of_files = []
      for file_name in os.listdir(root+folder+"/"):
        list_of_files.append(file_name)
      
      #  Shuffle the list
      rng.shuffle(list_of_files)

      # Create lists of files
      train_files = list_of_files[:int(len(list_of_files)*train_size)]
      test_files = list_of_files[int(len(list_of_files)*train_size) : int(len(list_of_files)*(train_size+test_size))]
      val_files = list_of_files[int(len(list_of_files)*(train_size+test_size)):]

      # Create folders and files for train data
      for one_file in train_files:
      
        # Copy  files
        dest_dir = "files/train/"+folder+"/"
        os.makedirs(dest_dir, exist_ok=True)

        shutil.copy2(src=(root+folder+"/"+one_file),
                    dst=(dest_dir+one_file))
      print(f"Folder {folder}. Train data copied. {len(train_files)} files")

      # Create folders and files for test data
      for one_file in test_files:      
        # Copy  files
        dest_dir = "files/test/"+folder+"/"
        os.makedirs(dest_dir, exist_ok=True)

        shutil.copy2(src=(root+folder+"/"+one_file),
                    dst=(dest_dir+one_file))
      print(f"Folder {folder}. Test data copied. {len(test_files)} files")

      # Create folders and files for validation data
      for one_file in val_files:
      
        # Copy  files
        dest_dir = "files/validation/"+folder+"/"
        os.makedirs(dest_dir, exist_ok=True)

        shutil.copy2(src=(root+folder+"/"+one_file),
                    dst=(dest_dir+one_file))
      print(f"Folder {folder}. Validation data copied. {len(val_files)} files")
      
     


def get_class_names_from_folder(directory):
  """
  Get the classnames from train folder for example
  """
  import pathlib
  import numpy as np
  data_dir = pathlib.Path(directory)
  class_names = np.array(sorted([item.name for item in data_dir.glob("*")])) # Created a list of class names 
  return class_names
  print(class_names)

In [3]:
# Split images dir to train, test and validation
split_dir_to_train_test_val(directory="data/",
                            train_size=0.78,
                            test_size=0.22,)

Folder kulfi. Train data copied. 104 files
Folder kulfi. Test data copied. 30 files
Folder kulfi. Validation data copied. 0 files
Folder jalebi. Train data copied. 151 files
Folder jalebi. Test data copied. 43 files
Folder jalebi. Validation data copied. 0 files
Folder burger. Train data copied. 181 files
Folder burger. Test data copied. 52 files
Folder burger. Validation data copied. 0 files
Folder pizza. Train data copied. 143 files
Folder pizza. Test data copied. 41 files
Folder pizza. Validation data copied. 0 files
Folder kadai_paneer. Train data copied. 177 files
Folder kadai_paneer. Test data copied. 51 files
Folder kadai_paneer. Validation data copied. 0 files
Folder pakode. Train data copied. 139 files
Folder pakode. Test data copied. 40 files
Folder pakode. Validation data copied. 0 files
Folder butter_naan. Train data copied. 169 files
Folder butter_naan. Test data copied. 48 files
Folder butter_naan. Validation data copied. 0 files
Folder pav_bhaji. Train data copied. 163 f

In [11]:
import tensorflow as tf

train_path = "files/train/"
test_path = "files/test/"

img_height = 180
img_width = 180

data_train = tf.keras.utils.image_dataset_from_directory(
    train_path,
    shuffle=True,
    image_size=(img_height, img_width),
    batch_size=32)

data_categs = data_train.class_names

data_val = tf.keras.utils.image_dataset_from_directory(
    test_path,
    shuffle=True,
    image_size=(img_height, img_width),
    batch_size=32)


print(data_categs)


Found 3106 files belonging to 20 classes.
Found 890 files belonging to 20 classes.
['burger', 'butter_naan', 'chai', 'chapati', 'chole_bhature', 'dal_makhani', 'dhokla', 'fried_rice', 'idli', 'jalebi', 'kaathi_rolls', 'kadai_paneer', 'kulfi', 'masala_dosa', 'momos', 'paani_puri', 'pakode', 'pav_bhaji', 'pizza', 'samosa']


In [12]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

model = Sequential([
    layers.Rescaling(1./255),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(units=len(data_categs)),
])

In [13]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [14]:
history = model.fit(
    data_train,
    validation_data=data_val,
    epochs=25,
)

Epoch 1/25
98/98 ━━━━━━━━━━━━━━━━━━━━ 36s 345ms/step - accuracy: 0.1086 - loss: 2.8743 - val_accuracy: 0.2506 - val_loss: 2.3964
Epoch 2/25
98/98 ━━━━━━━━━━━━━━━━━━━━ 41s 418ms/step - accuracy: 0.3352 - loss: 2.1758 - val_accuracy: 0.2775 - val_loss: 2.5480
Epoch 3/25
98/98 ━━━━━━━━━━━━━━━━━━━━ 37s 375ms/step - accuracy: 0.4692 - loss: 1.7491 - val_accuracy: 0.3888 - val_loss: 2.1021
Epoch 4/25
98/98 ━━━━━━━━━━━━━━━━━━━━ 41s 377ms/step - accuracy: 0.6485 - loss: 1.1628 - val_accuracy: 0.3247 - val_loss: 2.6038
Epoch 5/25
98/98 ━━━━━━━━━━━━━━━━━━━━ 41s 375ms/step - accuracy: 0.7946 - loss: 0.7158 - val_accuracy: 0.3854 - val_loss: 2.7923
Epoch 6/25
98/98 ━━━━━━━━━━━━━━━━━━━━ 38s 385ms/step - accuracy: 0.9276 - loss: 0.2599 - val_accuracy: 0.3416 - val_loss: 3.2084
Epoch 7/25
98/98 ━━━━━━━━━━━━━━━━━━━━ 37s 377ms/step - accuracy: 0.9688 - loss: 0.1138 - val_accuracy: 0.3843 - val_loss: 3.6310
Epoch 8/25
98/98 ━━━━━━━━━━━━━━━━━━━━ 37s 343ms/step - accuracy: 0.9605 - loss: 0.1292 - val_accu

In [15]:
model.save('ind.keras')

In [16]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('ind.tflite', 'wb') as f:
  f.write(tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmpx1n3gp54/assets


INFO:tensorflow:Assets written to: /tmp/tmpx1n3gp54/assets


Saved artifact at '/tmp/tmpx1n3gp54'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name='keras_tensor_11')
Output Type:
  TensorSpec(shape=(None, 20), dtype=tf.float32, name=None)
Captures:
  123729091001424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123729091000656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123729090995280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123729091002192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123729090999120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123729091000464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123729089855952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123729089856912: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1725695513.539613    5597 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1725695513.540443    5597 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-09-07 13:21:53.543890: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpx1n3gp54
2024-09-07 13:21:53.544642: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-09-07 13:21:53.544662: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpx1n3gp54
2024-09-07 13:21:53.556238: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-09-07 13:21:53.599387: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpx1n3gp54
2024-09-07 13:21:53.613564: I tensorflow/cc/saved_model/loader.cc:462] SavedModel load for tags { serve }; Status: success: OK. Took 69900 microseconds.


In [53]:
train_data.class_indices

{'burger': 0,
 'butter_naan': 1,
 'chai': 2,
 'chapati': 3,
 'chole_bhature': 4,
 'dal_makhani': 5,
 'dhokla': 6,
 'fried_rice': 7,
 'idli': 8,
 'jalebi': 9,
 'kaathi_rolls': 10,
 'kadai_paneer': 11,
 'kulfi': 12,
 'masala_dosa': 13,
 'momos': 14,
 'paani_puri': 15,
 'pakode': 16,
 'pav_bhaji': 17,
 'pizza': 18,
 'samosa': 19}